In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pandas.core.computation.expressions")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar

# Blasius ODE system
def blasius(eta, y):
    return [y[1], y[2], -0.5 * y[0] * y[2]]

# Shooting objective: f'(eta->∞) = 1
def objective(shoot_guess):
    y0 = [0, 0, shoot_guess]
    sol = solve_ivp(blasius, [0, 10], y0, t_eval=np.linspace(0, 10, 500))
    return sol.y[1, -1] - 1

# Find initial second derivative f''(0)
sol_root = root_scalar(objective, bracket=[0.3, 0.4], method='bisect')
fpp0 = sol_root.root

# Solve across eta
eta = np.linspace(0, 10, 200)
sol = solve_ivp(blasius, [0, 10], [0, 0, fpp0], t_eval=eta)
f, fp, fpp = sol.y

# Build DataFrame for values listing
df = pd.DataFrame({
    'eta': eta,
    'f':    f,
    "f'":   fp,
    "f''":  fpp
})

# Print values listing
print(df.to_string(index=False))

# Professional plot with white background and black lines
fig, ax = plt.subplots(figsize=(8, 6), facecolor='white')
ax.plot(eta, f,   label="f",   color='black', linewidth=2)
ax.plot(eta, fp,  label="f'",  color='black', linewidth=2, linestyle='--')
ax.plot(eta, fpp, label="f''", color='black', linewidth=2, linestyle='-.')

# White figure and axes background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Remove top and right spines
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

# Ticks and grid
ax.tick_params(direction='out', length=6, width=1)
ax.grid(True, linestyle=':', linewidth=0.5)

# Axes labels and legend
ax.set_xlabel(r'$\eta$', fontsize=14)
ax.set_ylabel('Solution profiles', fontsize=14)
ax.legend(frameon=False, fontsize=12)

plt.tight_layout()
plt.savefig('Blasius.png', dpi=300, format='png') 
plt.show()
